In [1]:
import argparse
import os
from typing import List
from typing import Optional

import lightning.pytorch as pl
import optuna
from optuna.integration import PyTorchLightningPruningCallback
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision import datasets
from torchvision import transforms

In [6]:
class LightningNet(pl.LightningModule):
    def __init__(self, dropout: float, output_dims: List[int]):
        super().__init__()
        self.model = Net(dropout, output_dims)

    def forward(self, data: torch.Tensor) -> torch.Tensor:
        return self.model(data.view(-1, 28 * 28))

    def training_step(self, batch, batch_idx: int) -> torch.Tensor:
        data, target = batch
        output = self(data)
        return F.nll_loss(output, target)

    def validation_step(self, batch, batch_idx: int) -> None:
        data, target = batch
        output = self(data)
        pred = output.argmax(dim=1, keepdim=True)
        accuracy = pred.eq(target.view_as(pred)).float().mean()
        self.log("val_acc", accuracy, sync_dist=True)
        self.log("hp_metric", accuracy, on_step=False, on_epoch=True, sync_dist=True)

    def configure_optimizers(self) -> optim.Optimizer:
        return optim.Adam(self.model.parameters())


In [7]:
class FashionMNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str, batch_size: int):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage: Optional[str] = None) -> None:
        self.mnist_test = datasets.FashionMNIST(
            self.data_dir, train=False, download=True, transform=transforms.ToTensor()
        )
        mnist_full = datasets.FashionMNIST(
            self.data_dir, train=True, download=True, transform=transforms.ToTensor()
        )
        self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.mnist_train, batch_size=self.batch_size, shuffle=True, pin_memory=True
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.mnist_val, batch_size=self.batch_size, shuffle=False, pin_memory=True
        )

    def test_dataloader(self) -> DataLoader:
        return DataLoader(
            self.mnist_test, batch_size=self.batch_size, shuffle=False, pin_memory=True
        )

In [1]:
import sys
import os
from glob import glob 
import pandas as pd

sys.path.append(os.pardir)
from src.config import *
from src.dataset import DataModule, ImageDataset

In [2]:
img_paths = glob(f'{DATA_DIR}/images/*')
df = pd.read_csv(f'{DATA_DIR}/verified_annotation_from_xml.csv')

df = pd.read_csv(f'{DATA_DIR}/verified_annotation_from_xml.csv')
df['img_path'] =f'{DATA_DIR}/images/' + df['image_name']
df.drop(columns=['Unnamed: 0'], inplace=True)
df['label_name'] = df['label_name'].apply(lambda x: x.lower())
df['sparse_label'] = df['label_name'].map({'atopic': 0, 'papular': 1,'scabies': 2})
df.head()


,label_name,xmin,ymin,xmax,ymax,image_name,image_width,image_height,patient_id,img_path,sparse_label
0,papular,0,919,1635,2918,00241a_A_P.jpg,1960,4032,3.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,1
1,scabies,1006,1260,1713,2143,00240b_A_S.jpg,1960,4032,6.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,2
2,atopic,215,424,457,706,0052b_A_A.jpg,720,1280,9.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,0
3,scabies,138,94,633,643,0044a_A_S.jpg,1280,1280,14.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,2
4,scabies,354,866,950,1217,0044a_A_S.jpg,1280,1280,14.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,2


In [4]:
import cv2
import pandas as pd
from typing import Optional
from sklearn.model_selection import GroupShuffleSplit

import torch
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from .config import *


class ImageDataset(Dataset):
    def __init__(self, df, input_shape, transform=None):
        self.df = df
        self.transform = transform
        self.input_shape = input_shape

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'img_path']
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        xmin, ymin, xmax, ymax = self.df.loc[idx, ['xmin', 'ymin', 'xmax', 'ymax']].values
        xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)

        img = img[ymin:ymax, xmin:xmax]
        img = cv2.resize(img, self.input_shape[:-1], interpolation=cv2.INTER_LINEAR)

        if self.transform:
            img = self.transform(img)

        sparse_label = int(self.df.loc[idx, 'sparse_label'])
        sparse_label_tensor = torch.tensor(sparse_label)

        cat_label = F.one_hot(sparse_label_tensor, num_classes=NUM_CLASSES)
        return torch.from_numpy(img.transpose((2, 0, 1))), cat_label
    

NUM_CLASS = 3
BATCH_SIZE = 3
class DataModule(pl.LightningDataModule):
    def __init__(self, model_name: str) -> None:
        super().__init__()
        self.resize_size = RESIZE_SIZE[model_name][:-1]
        self.crop_size = CROP_SIZE[model_name][:-1]
        self.batch_size = BATCH_SIZE
        self.num_workers = NUM_WORKERS
        # self.prepare_data()
        # self.setup() 


    def prepare_data(self) -> None:
        self.df = pd.read_csv(f'{DATA_DIR}/verified_annotation_from_xml.csv')
        self.df['img_path'] =f'{DATA_DIR}/images/' + self.df['image_name']
        self.df.drop(columns=['Unnamed: 0'], inplace=True)
        self.df['label_name'] = self.df['label_name'].apply(lambda x: x.lower())
        self.df['sparse_label'] = self.df['label_name'].map({'atopic': 0, 'papular': 1,'scabies': 2})

    def setup(self, stage: Optional[str] = None) -> None:

        gs = GroupShuffleSplit(n_splits=2, train_size=.85, random_state=42)

        train_val_idx, test_idx = next(gs.split(self.df,groups=self.df.patient_id))
        train_val_df = self.df.iloc[train_val_idx]
        test_df = self.df.iloc[test_idx]

        train_idx, val_idx = next(gs.split(train_val_df, groups=train_val_df.patient_id))
        train_df = train_val_df.iloc[train_idx]
        val_df = train_val_df.iloc[val_idx]

        train_df.reset_index(drop=True, inplace=True)
        val_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        self.train_ds = ImageDataset(
            df=train_df, 
            input_shape=self.resize_size, 
            transform=transforms.Compose(
                [
                    transforms.ToPILImage(),
                    # transforms.Resize(self.resize_size),
                    transforms.CenterCrop(self.crop_size),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomVerticalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                ]
            )
        )
        self.val_ds = ImageDataset(
            df=val_df, 
            input_shape=self.resize_size, 
            transform=transforms.Compose(
                [
                    transforms.ToPILImage(),
                    # transforms.Resize(self.resize_size),
                    transforms.CenterCrop(self.crop_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                ]
            )
        )

        self.test_ds = ImageDataset(
            df=test_df, 
            input_shape=self.resize_size, 
            transform=transforms.Compose(
                [
                    transforms.ToPILImage(),
                    # transforms.Resize(self.resize_size),
                    transforms.CenterCrop(self.crop_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                ]
            )
        )

    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True,
            pin_memory=True
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.val_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            pin_memory=True
        )

    def test_dataloader(self) -> DataLoader:
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            pin_memory=True
        )   

ImportError: attempted relative import with no known parent package

In [3]:
data_module = DataModule(model_name='efficientnet_b0')


NameError: name 'RESIZE_SIZE' is not defined

In [3]:
data_module = DataModule(model_name='efficientnet_b0')
for x, y in data_module.train_dataloader():
    print(x.shape, y.shape)